Travelling Salesperson Problem
---

A classic problem in operations research. See eg Norvig, http://nbviewer.ipython.org/url/norvig.com/ipython/TSP.ipynb, for a great treatment.

We will represent a salesperson's tour as a list of integers, each integer an index into a list of cities. Of course, the integers must form a permutation.

First, we define some generators for comparison. We will demonstrate two new ideas:

1. Generators which call subroutines.
2. Using the problem instance in a generator.

### Generators which call subroutines

As before, we will import a `random` object in which random calls are instrumented for tracing. This time, our generators will call some subroutines which themselves make random calls. We have to inform PTO that these subroutines should be recorded in the program trace. We will do so by importing a `random_function` decorator, and applying it to each of the relevant subroutines.

(In the simple cases *onemax* and *sphere*, the generator itself made random calls, but there were no subroutines. PTO knows that the generator itself should always be recorded, so in those simple cases, we didn't need to use the `random_function` decorator. By the way, if we decorate a subroutine that doesn't need to be recorded, nothing bad happens. But it would be confusing for other readers.)

### Using the problem instance in a generator

In `sphere`, we saw that some problems can be defined parametrically, with a *problem size* parameter, and when we want to study scalability across problem sizes, PTO provides a way to do that.

Next, we'll observe that some problems are naturally parametrized in terms of a *problem instance*. The problem instance is composed of any problem-specific data, which may include problem *size* (thus instance is a generalisation of size). In PTO, it is natural for generators to look at the problem instance to influence how the make their random decisions. So, we will write some generators which take the problem instance as an argument. The problem instance will be a square city-city distance matrix named `inst`.

In [1]:
from PTO import random, random_function, solve

# no need for @random_function because this will be used as a generator itself
def randsol1(inst):
    # Create a permutation by shuffling. We provide a custom shuffle function.
    return swap_shuffle(list(range(len(inst))))

@random_function # inform PTO that this function must be traced
def swap_shuffle(perm): 
    for i in range(len(perm)):
        ri = random.choice(range(i,len(perm)))
        perm[i],perm[ri]=perm[ri],perm[i]
    return perm

# no need for @random_function because this will be used as a generator itself
def randsol2(inst):
    # Create a permutation by shuffling. We provide a custom shuffle function.
    return rev_shuffle(list(range(len(inst))))

@random_function # inform PTO that this function must be traced
def rev_shuffle(perm):
    # this is like multiple applications of 2-opt
    for i in range(len(perm)):
        ri = random.choice(range(i,len(perm)))
        perm[i:ri+1] = perm[i:ri+1][::-1] # reverse a section
    return perm

### *Really* using the problem instance

The two generators above look at the problem instance, but don't use it in any meaningful way. The next generator does: it builds up a permutation step by step, at each step choosing a city which is near the current one, with high probability. This is similar to the main idea of GRASP (Ribeiro and Resende) and the biased randomisation heuristic (Juan et al.).

In [2]:
def randsol3(inst):
    """Take advantage of problem data in the simplest possible way"""
    n = len(inst)
    sol = []
    remaining = list(range(n))

    # the start city is a decision variable, because we'll get different results if
    # we start at different cities

    x = random.choice(list(range(n))) #(remaining) THIS WAS THE ROOT OF THE ERROR. 
    # Partial + Mutable in Local + argument of wrapped function = weird side effect... just avoid this combination!

    sol.append(x)
    remaining.remove(x)

    i = 1
    while i < n:

        # choose one of the remaining cities randomly, weighted by
        # inverse distance.
        x = choose_node(inst, x, remaining)
        sol.append(x)
        remaining.remove(x)

        i += 1

    return sol

# no need for @random_function, because choose_node makes no random calls itself
def choose_node(inst, cur, remaining):
    wts = [1.0 / inst[cur][n] for n in remaining]
    s = sum(wts)
    wts = [wt / float(s) for wt in wts] # normalise
    return roulette_wheel(remaining, wts)

@random_function # inform PTO that this function must be traced
def roulette_wheel(items, wts): # assumes wts sum to 1
    x = random.random()
    for item, wt in zip(items, wts):
        x -= wt
        if x <= 0:
            return item
    # Should not reach here
    print("Error")
    print(items)
    print(wts)
    raise ValueError

### Fitness

Our generators take an `inst` (instance) argument. Our fitness function will too. Fitness is the negative of tour length, given a permutation `perm` and a city-city distance matrix `inst`.

In [3]:
def fitness(perm, inst):
    # note negative indexing trick to include final step (return to origin)
    return -sum([inst[perm[i-1]][perm[i]] for i in range(0,len(perm))])

### Random problem instances

We will generate random instances, given a problem size (number of cities) `n`, by generating a distance matrix with values exponentially distributed.

(Note also that the use of `random` here will not be recorded by the tracer (see *Generators that call subroutines* above), as intended, because it will run _before_ calling the solver.)

In [4]:
def randprob(n): 
    c = [[0 for x in range(n)] for x in range(n)] # initialise cost matrix
    for i in range(n):
        for j in range(n):
            if i == j:
                c[i][j] = 0 # zero diagonal
            elif i > j:
                c[i][j] = c[j][i] # symmetric matrix
            else:
                c[i][j] = random.expovariate(1) # more interesting than uniform weights
    return c

Now we can generate a problem instance and solve it.

The extra `inst` argument in both the generators and the fitness function parallel the extra `problem_size` argument in `sphere.ipynb`. As in that case, if we want to use `solve`, we have to specialise these functions to a particular instance. So, we generate the instance and then use `functools.partial`.

In [5]:
from functools import partial

inst = randprob(10) # generate a toy instance, represented by a city-city distance matrix
randsol1_inst = lambda: randsol1(inst) # specialise the generator for that instance
fitness_inst = lambda x: fitness(x, inst) # specialise the fitness for that instance

ind, fit = solve(randsol1_inst, fitness_inst, solver="MGA", str_trace=True, effort=1.0)
print(ind)
print(fit)

[7, 3, 0, 9, 4, 5, 6, 8, 1, 2]
-5.296325149679174


Experiments and Analysis
---

Next, we can do a large run and carry out some analysis on our results. 
We'll import functions provided by PTO, and also use scipy. `compare_all` is able to generate random problem instances internally, and specialise the fitness function and generator, so we pass in the original (not specialised) versions of these.

In [6]:
from PTO import compare_all, stat_summary, make_table
import scipy.stats # only for post-run analysis

In [7]:
results = compare_all(fitness, 
                      [randsol1, randsol2], # multiple generators
                      sizes=[20],
                      random_instance=randprob, # new random instance for each run
                      methods=["MGA"],
                      str_traces=[False, True], # multiple trace types
                      budget=10000, 
                      num_runs=15)

**Experiment 1**: Compare structured trace with linear, using hill-climbing: no difference, using any generator.

In [8]:
d0, d1 = results[(20, False, 'randsol1', 'MGA')], results[(20, True, 'randsol1', 'MGA')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean -10.42 std 1.69 min -14.45 med -10.19 max -7.21
mean -10.83 std 1.31 min -12.70 med -11.04 max -8.44
Ttest_indResult(statistic=0.7201652429332626, pvalue=0.4773925411787284)


In [9]:
d0, d1 = results[(20, False, 'randsol2', 'MGA')], results[(20, True, 'randsol2', 'MGA')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean -10.77 std 1.52 min -13.76 med -11.08 max -7.95
mean -10.67 std 1.14 min -13.19 med -10.35 max -9.05
Ttest_indResult(statistic=-0.2050267362180771, pvalue=0.839034440635087)


In [10]:
d0, d1 = results[(20, False, 'randsol3', 'HC')], results[(20, True, 'randsol3', 'HC')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

C:\Users\conneljo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\conneljo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\conneljo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\conneljo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\conneljo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity

**Experiment 2**: Compare one generator against another, using structured trace. Some difference between randsol1 and 2, but large improvements with randsol3 (the one which uses the problem data).

In [11]:
d0, d1 = results[(20, True, 'randsol1', 'HC')], results[(20, True, 'randsol2', 'HC')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

ValueError: zero-size array to reduction operation minimum which has no identity

In [20]:
d0, d1 = results[(20, True, 'randsol1', 'HC')], results[(20, True, 'randsol3', 'HC')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean -4.66 std 0.80 min -6.06 med -4.59 max -3.11
mean -2.71 std 0.65 min -4.14 med -2.46 max -1.85
Ttest_indResult(statistic=-7.0707106203998018, pvalue=1.0828123959424254e-07)


In [21]:
d0, d1 = results[(20, True, 'randsol2', 'HC')], results[(20, True, 'randsol3', 'HC')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean -6.91 std 1.28 min -9.41 med -7.12 max -4.86
mean -2.71 std 0.65 min -4.14 med -2.46 max -1.85
Ttest_indResult(statistic=-10.925473805613871, pvalue=1.3169140516706127e-11)


### Some real TSP instances: TSPLIB

Up to now, we've been generating random TSP instances to demonstrate investigation of scalability. Next, we'll read some real TSP instances from TSPLIB.

First we get TSPLIB itself. If the following code fails for any reason, just download `ALL_tsp.tar.gz` from the given URL, and extract it into a new directory `/TSPLIB` in the same directory as this notebook.

In [22]:
import os.path

dirname = "TSPLIB"
def get_TSPLIB(dirname):
    import os
    import tarfile
    import requests # conda install requests, or pip install requests

    tsplib_url = "http://www.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp/ALL_tsp.tar.gz"
    r = requests.get(tsplib_url)
    os.makedirs(dirname)
    filename = os.path.join(dirname, "ALL_tsp.tar.gz")
    f = open(filename, "wb")
    f.write(r.content)
    f.close()
    tar = tarfile.open(filename, "r:gz")
    tar.extractall(dirname)
    tar.close()
    
if not os.path.isfile(os.path.join(dirname, "att48.tsp.gz")): # example to check if TSPLIB already exists
    get_TSPLIB(dirname)

ReadError: not a gzip file

In [36]:
import math, gzip

class TSP:
    def __init__(self, filename):
        self.cities = []
        self.read_file(filename)
        self.matrix = [[self.euclidean_distance(city_i, city_j) 
                        for city_i in self.cities] 
                       for city_j in self.cities]
        # self.read_optimal_results("TSPLIB/STSP.html")

    def euclidean_distance(self, x, y):
        return math.sqrt(sum((xi - yi) ** 2.0 for xi, yi in zip(x, y)))
    
    def read_optimal_results(self, filename):
        # If we would like to look at known optima for these problem see
        # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html. Put
        # that .html file under TSPLIB.
        import re
        optimal_results = {}
        for line in open(filename).readlines():
            p = r">(\w+) : (\d+)<"
            m = re.search(p, line)
            if m:
                key, val = m.group(1, 2)
                key = key.strip()
                # optimal results are given as integers in TSPLIB
                val = int(val.split()[0].strip())
                optimal_results[key] = val
        # print("Optimal results:")
        # print(optimal_results)
        self.optimal = optimal_results[self.name]
        print("Optimum: " + str(self.optimal))

    def read_file(self, filename):
        """FIXME this only works for files in the node xy-coordinate
        format. Some files eg bayg29.tsp.gz give explicit edge weights
        instead."""
        f = gzip.open(filename, "rt") # note t to force text mode
        coord_section = False
        for line in f.readlines():
            line = line.strip()
            if line.startswith("NAME"):
                self.name = line.split(":")[1].strip()
            elif (line.startswith("COMMENT") or
                  line.startswith("TYPE") or
                  line.startswith("EDGE_WEIGHT_TYPE")):
                pass
            elif line.startswith("DIMENSION"):
                self.n = int(line.split(":")[1].strip())
            elif line.startswith("NODE_COORD_SECTION"):
                coord_section = True
            elif line.startswith("EOF"):
                break
            elif coord_section:
                # coords are sometimes given as floats in TSPLIB
                idx, x, y = map(float, line.split(" "))
                self.cities.append((x, y))

        
filenames = ["att48.tsp.gz", "berlin52.tsp.gz"]
insts = [TSP(os.path.join(dirname, filename)).matrix for filename in filenames]

### `compare_all` with pre-made instances

Now we have a set of problem instances on which we want to test. `compare_all` can handle that case too: we pass them in with `instances=[...]`, and omit the `sizes` and `random_instance` arguments.

In [38]:
results = compare_all(fitness, 
                      [randsol1, randsol3], # multiple generators
                      instances=insts,
                      methods=["HC"],
                      budget=10000, 
                      num_runs=15)

We see that for both of these problems (0 and 1 below), `randsol3` (using the GRASP-like problem data approach) wins:

In [39]:
make_table(results)

(0, True, 'randsol1', 'HC'): mean -69967.14 std 6623.34 min -86377.22 med -67960.97 max -62804.23
(0, True, 'randsol3', 'HC'): mean -62941.83 std 5384.10 min -75867.82 med -60907.97 max -55909.58
(1, True, 'randsol1', 'HC'): mean -15344.31 std 1141.77 min -18103.86 med -15109.93 max -13342.80
(1, True, 'randsol3', 'HC'): mean -14470.33 std 1136.77 min -15809.94 med -14837.14 max -11821.65


Conclusions
---

We have demonstrated the use of a permutation search space, and the comparison of multiple generators, and the use of problem data in the generator for improved performance. And we have seen some more use of `compare_all` and statistical testing on its results, and how to read problem instances for TSP.

